In [1]:
file_name="NER/output2.txt" # Define data file path
import codecs
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
from nltk.tokenize import RegexpTokenizer
import glob
import nltk
import re
import string
import pandas as pd

## Create necessary function

In [2]:
#จัดการประโยคซ้ำ
data_not=[]
def Unique(p):
    text=re.sub("<[^>]*>","",p)
    text=re.sub("\[(.*?)\]","",text)
    text=re.sub("\[\/(.*?)\]","",text)
    if text not in data_not:
        data_not.append(text)
        return True
    else:
        return False

In [3]:
# เตรียมตัวตัด tag ด้วย re
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัดคำ
# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
    text=text.strip()
    text=re.sub("<[^>]*>","",text)
    text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')#  ตัดการกับพวกไม่มี tag word
    text=re.sub("(\[\w+\])","***\\1",text)
    text2=[]
    for i in text.split('***'):
        if "[" in i:
            text2.append(i)
        else:
            text2.append("[word]"+i+"[/word]")
    text="".join(text2)
    return text.replace("[word][/word]","")

In [4]:
def text2conll2002(text,pos=True):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text) # นำไปใส่ tag [word]
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer.tokenize(text) # แยก tag ออกมาจากข้อความ
    j=0
    conll2002="" # ประกาศตัวแปรเก็บ conll2002
    for tagopen,text,tagclose in tag: # ลูปใน tag โดยเป็น (tagopen,text,tagclose)
        word_cut=word_tokenize(text,engine="newmm") # ใช้ตัวตัดคำ newmm ของ PyThaiNLP
        i=0
        txt5=""
        while i<len(word_cut): #ลูปตามจำนวน token ที่ตัดในtag
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word': # ไม่เป็น tag [word] และเป็น i หรือตัวเริ่มต้น tag
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen
            else: # เป็น [word]
                txt5+=word_cut[i]
                txt5+='\t'+'O'
            txt5+='\n'
            #j+=1
            i+=1
        conll2002+=txt5
    if pos==False:
        return conll2002
    return postag(conll2002) # เพิ่ม postag ใส่
# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
# print(text2conll2002(t,pos=False))

In [5]:
def postag(text):
    listtxt=[i for i in text.split('\n') if i!='']
    list_word=[]
    for data in listtxt:
        list_word.append(data.split('\t')[0])
#     print("postag")    
#     print(list_word)
    list_word=pos_tag(list_word,engine="perceptron", corpus='orchid')
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\n'
        i+=1
    return text

In [6]:
# อ่านข้อมูลจากไฟล์
def get_data(fileopen):
    """
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
    with codecs.open(fileopen, 'r',encoding='utf-8-sig') as f:
        lines = f.read().splitlines()
    return [a for a in lines if Unique(a)] # เอาไม่ซ้ำกัน
def alldata(lists):
    text=""
    for data in lists:
        text+=text2conll2002(data)
        text+='\n'
    return text

def alldata_list(lists):
    data_all=[]
    for data in lists:
        data_num=[]
        try:
            txt=text2conll2002(data,pos=True).split('\n') # นำไปแปลงเป็น conll2002
            for d in txt:
                tt=d.split('\t')
                if d!="":
                    if len(tt)==3:
                        data_num.append((tt[0],tt[1],tt[2]))
                    else:
                        data_num.append((tt[0],tt[1]))
            #print(data_num)
            data_all.append(data_num)
        except:
            print(data)
    #print(data_all)
    return data_all

def alldata_list_str(lists):
	string=""
	for data in lists:
		string1=""
		for j in data:
			string1+=j[0]+"	"+j[1]+"	"+j[2]+"\n"
		string1+="\n"
		string+=string1
	return string

def get_data_tag(listd):
	list_all=[]
	c=[]
	for i in listd:
		if i !='':
			c.append((i.split("\t")[0],i.split("\t")[1],i.split("\t")[2]))
		else:
			list_all.append(c)
			c=[]
	return list_all
def getall(lista):
    ll=[]
    for i in lista:
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(i)
    return ll

## Getting and preparing data set

In [7]:
train_data=getall(get_data(file_name)) # นำคลังเข้าไป แยกออกเป็น list ละบรรทัด

In [8]:
len(train_data)

2443

In [9]:
train_data

['ไปๆๆๆๆ',
 'ไปๆๆ[RESTAURANT]mk[/RESTAURANT] ',
 'ไป',
 'กินที่ไหน',
 'กินอันนี้',
 'ของชอบใคร',
 'ขอบคุณค่ะ',
 'ขอบคุณจ้า',
 'ไข่ม้วน',
 'ค่ดน่ากิน',
 'เคค รอแปป',
 'จะทำไรอ่ะ',
 'จัดๆอยากๆ',
 'น่าโดน',
 'น่าไปลองอะ',
 'น่าสนๆ',
 'นี่เลย 5',
 '[CAR]บริโอ้[/CAR]  สวย',
 'รักกกกก',
 'รีบจัดสิฮะ',
 'เรากิน[GOODS]ช้าง[/GOODS] ',
 'ลงทีใหนคับ',
 'เลี้ยงกุไป',
 'วันนี้ปะๆๆ',
 'วันไหนก็ดี',
 'วันไหนดีคะ',
 'ว้าวววว',
 'ศ นี้เลยนะ',
 'สน [CAR]มาสด้า3[/CAR] ',
 'สนใจ[CAR]mg3[/CAR] ค่ะ',
 'สักเมามั้ย',
 'สักวันเถอะ',
 'หิวใช่มั้ย',
 'หิว[GOODS]เป็ด[/GOODS] จุง',
 'หิวแล้วววว',
 'อยากกิน [RESTAURANT]mk[/RESTAURANT] ',
 'อยากกินๆๆๆ',
 'อยากกินค่ะ',
 'อยากไปปปปป',
 'อยากอิเห้ย',
 'อยาก[RESTAURANT]ฮอตพอท[/RESTAURANT] ',
 'อยากิน[GOODS]ลีโอ[/GOODS] ',
 'อ๊ะ ไปก็ไป',
 'อะไรเอ่ยยย',
 'อิ่มแล้ว',
 '[RESTAURANT]เอ็มเค[/RESTAURANT] เถอะ',
 'เออดูอีกที',
 'โอ้ยยยอยาก',
 'อ่ะได้ม่ะ',
 'fc [CAR]นาวาร่า[/CAR] ',
 'good idea',
 'กลางวันนนี้',
 'กินๆๆ[RESTAURANT]เอ็มเค[/RESTAURANT] ',
 'กินกะหนูไงง',
 'กินเก่ง

Pos tagging the data set

In [10]:
datatofile=alldata_list(train_data) # นำไปผ่านขั้นตอน 1 2 3 4

In [11]:
datatofile

[[('ไป', 'XVAE', 'O'), ('ๆๆๆๆ', 'NCMN', 'O')],
 [('ไป', 'XVAE', 'O'), ('ๆๆ', 'NCMN', 'O'), ('mk', 'NCMN', 'B-RESTAURANT')],
 [('ไป', 'XVAE', 'O')],
 [('กิน', 'VACT', 'O'), ('ที่ไหน', 'PNTR', 'O')],
 [('กิน', 'VACT', 'O'), ('อันนี้', 'ADVP', 'O')],
 [('ของ', 'RPRE', 'O'), ('ชอบ', 'NCMN', 'O'), ('ใคร', 'PNTR', 'O')],
 [('ขอบคุณ', 'VACT', 'O'), ('ค่ะ', 'NCMN', 'O')],
 [('ขอบคุณ', 'VACT', 'O'), ('จ้า', 'NCMN', 'O')],
 [('ไข่', 'NCMN', 'O'), ('ม้วน', 'ADVN', 'O')],
 [('ค่ด', 'NCMN', 'O'), ('น่ากิน', 'ADVN', 'O')],
 [('เค', 'NCMN', 'O'),
  ('ค', 'NCMN', 'O'),
  (' ', 'PUNC', 'O'),
  ('รอ', 'VSTA', 'O'),
  ('แป', 'NCMN', 'O'),
  ('ป', 'NCMN', 'O')],
 [('จะ', 'XVBM', 'O'),
  ('ทำ', 'VACT', 'O'),
  ('ไร', 'NCMN', 'O'),
  ('อ่ะ', 'NCMN', 'O')],
 [('จัด', 'VACT', 'O'),
  ('ๆ', 'NCMN', 'O'),
  ('อยาก', 'XVMM', 'O'),
  ('ๆ', 'NCMN', 'O')],
 [('น่า', 'NCMN', 'O'), ('โดน', 'NCMN', 'O')],
 [('น่า', 'VSTA', 'O'),
  ('ไป', 'XVAE', 'O'),
  ('ลอง', 'VACT', 'O'),
  ('อะ', 'ADVP', 'O')],
 [('น่าสน', 'NCMN',

In [12]:
tt=[]
with open("NER/train_set-pos.conll","w",encoding="utf8") as f:
    i=0
    while i<len(datatofile):
        for j in datatofile[i]:
            f.write(j[0]+"\t"+j[1]+"\t"+j[2]+"\n")
        if i+1<len(datatofile):
            f.write("\n")
        i+=1

with open("NER/train_set.conll","w",encoding="utf8") as f:
    i=0
    while i<len(datatofile):
        for j in datatofile[i]:
            f.write(j[0]+"\t"+j[2]+"\n")
        if i+1<len(datatofile):
            f.write("\n")
        i+=1
from sklearn_crfsuite import scorers,metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate,train_test_split
import sklearn_crfsuite
def doc2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    # Features from current word
    features={
        'word.word': word,
        'word.isspace':word.isspace(),
        'postag':postag,
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        prevword = doc[i-1][0]
        postag1 = doc[i-1][1]
        features['word.prevword'] = prevword
        features['word.previsspace']=prevword.isspace()
        features['word.prepostag'] = postag1
        features['word.prevwordisdigit'] = prevword.isdigit()
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        postag1 = doc[i+1][1]
        features['word.nextword'] = nextword
        features['word.nextisspace']=nextword.isspace()
        features['word.nextpostag'] = postag1
        features['word.nextwordisdigit'] = nextword.isdigit()
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features

def extract_features(doc):
    return [doc2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [tag for (token,postag,tag) in doc]


X_data = [extract_features(doc) for doc in datatofile] # เอา คำ แยกออกมา
y_data = [get_labels(doc) for doc in datatofile] # เอา tag แยกออกมา

In [13]:
len(X_data)

2443

### Split data

Split data to train and test(valadate) set with ratio [75:25] = [train:test]

In [14]:
X, X_test, y, y_test = train_test_split(X_data, y_data, test_size=0.25) # แบ่ง 0.1 หรือ 10%

In [15]:
len(X)

1832

In [16]:
len(X_test)

611

In [17]:
len(y)

1832

In [18]:
len(y_test)

611

## Modeling and compilation

In [19]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=500,
    all_possible_transitions=True,
    model_filename=file_name+"-pos.model0" # ตั้งชื่อโมเดล
)

In [20]:
crf.fit(X, y); # train

## Evaluate the performance of the model

In [21]:
labels = list(crf.classes_)
labels.remove('O')
y_pred = crf.predict(X_test)
e=metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)
# Show the performance 
print(e)

C:\Users\nuttapol\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\nuttapol\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.8133802627791291


In [22]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

C:\Users\nuttapol\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\nuttapol\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


               precision    recall  f1-score   support

        B-APP      0.000     0.000     0.000         0
     B-ARTIST      0.000     0.000     0.000         1
     I-ARTIST      0.000     0.000     0.000         1
        B-CAR      0.902     0.804     0.851       138
        I-CAR      0.909     0.725     0.806       138
   B-COSMETIC      0.486     0.692     0.571        26
   I-COSMETIC      0.000     0.000     0.000        13
      B-GOODS      0.934     0.807     0.866        88
      I-GOODS      1.000     0.581     0.735        43
   B-LOCATION      0.000     0.000     0.000         2
     B-PERSON      0.000     0.000     0.000         2
     I-PERSON      0.000     0.000     0.000         8
     B-PRESON      0.000     0.000     0.000         0
     I-PRESON      0.000     0.000     0.000         0
 B-RESTAURANT      0.936     0.912     0.924        80
 I-RESTAURANT      0.952     0.952     0.952       105
   B-SKINCARE      1.000     0.556     0.714         9
   I-SKIN

In [23]:
sorted_labels

['B-APP',
 'B-ARTIST',
 'I-ARTIST',
 'B-CAR',
 'I-CAR',
 'B-COSMETIC',
 'I-COSMETIC',
 'B-GOODS',
 'I-GOODS',
 'B-LOCATION',
 'B-PERSON',
 'I-PERSON',
 'B-PRESON',
 'I-PRESON',
 'B-RESTAURANT',
 'I-RESTAURANT',
 'B-SKINCARE',
 'I-SKINCARE',
 'B-SUPERMARKET']

In [24]:
# Create function for Name Entity Regcognition use.
def get_ner(text):
    word_cut=word_tokenize(text.lower(),engine="newmm")
    list_word=pos_tag(word_cut,engine='perceptron', corpus='orchid')
    X_test = extract_features([(data,list_word[i][1]) for i,data in enumerate(word_cut)])
    y_=crf.predict_single(X_test)
    return [(word_cut[i],list_word[i][1],data) for i,data in enumerate(y_)]

In [25]:
# while True:
#     t=input("Text : ")
#     print(get_ner(t))

In [26]:
text='วันพรุ่งนี้อยากไปกินเป็ดย่างเอ็มเค'

In [27]:
get_ner(text)

[('วันพรุ่งนี้', 'JSBR', 'O'),
 ('อยาก', 'XVMM', 'O'),
 ('ไป', 'VACT', 'O'),
 ('กิน', 'VACT', 'O'),
 ('เป็ด', 'NCMN', 'O'),
 ('ย่าง', 'VATT', 'O'),
 ('เอ็ม', 'NCMN', 'B-RESTAURANT'),
 ('เค', 'NCMN', 'I-RESTAURANT')]

In [28]:
print(get_ner(text))

[('วันพรุ่งนี้', 'JSBR', 'O'), ('อยาก', 'XVMM', 'O'), ('ไป', 'VACT', 'O'), ('กิน', 'VACT', 'O'), ('เป็ด', 'NCMN', 'O'), ('ย่าง', 'VATT', 'O'), ('เอ็ม', 'NCMN', 'B-RESTAURANT'), ('เค', 'NCMN', 'I-RESTAURANT')]


In [29]:
pos_conll = pd.read_csv('./NER/train_set-pos.conll',header = None, sep = "\t")

In [30]:
pos_conll

,0,1,2
0,ไป,XVAE,O
1,ๆๆๆๆ,NCMN,O
2,ไป,XVAE,O
3,ๆๆ,NCMN,O
4,mk,NCMN,B-RESTAURANT
5,ไป,XVAE,O
6,กิน,VACT,O
7,ที่ไหน,PNTR,O
8,กิน,VACT,O
9,อันนี้,ADVP,O


In [31]:
test_pos = [line.strip() for line in open("sentiment-assignment/pos.txt", 'r', encoding="utf8")]
test_neg = [line.strip() for line in open("sentiment-assignment/neg.txt", 'r', encoding="utf8")]
test_neu = [line.strip() for line in open("sentiment-assignment/neu.txt", 'r', encoding="utf8")]

In [32]:
test_set = test_pos+test_neg+test_neu

In [33]:
test_data = [get_ner(sentence) for sentence in test_set]

In [34]:
len(test_data)

27506

In [35]:
test_data[99]

[('ชอบ', 'NCMN', 'O'), ('อะ', 'NCMN', 'O')]

In [36]:
for word in test_data[99]:
    print(word[2][0])

O
O


In [37]:
found_word=[]
for sentence in test_data:
    for idx in range(len(sentence)):
        if (idx+1) < len(sentence):
            if sentence[idx][2][0] == 'B' and sentence[idx+1][2][0] == 'I':
                temp=sentence[idx][0]+sentence[idx+1][0]
                found_word.append(temp)
            elif  sentence[idx][2][0] == 'B' and sentence[idx+1][2][0] != 'I':
                found_word.append(sentence[idx][0])

In [38]:
found_word

['เอมเค',
 'มาสด้า3',
 'ฮอตพอ',
 'นาวาร่า',
 'เอ็มเค',
 'mg3',
 'บาบีก้อน',
 'มาสด้าสาม',
 'แคปติ',
 'บาบีกอ',
 'บาบีก้อน',
 'บาบีก้อน',
 'บาร์บี',
 'ช้าง',
 'บาบีก้อน',
 'เอ็มเค',
 'ฮอตพอ',
 'nissan<full_stop>',
 'ช้างจ่าย',
 'ฮอทพอ',
 'ดีแมก',
 'ช้างวม',
 'honda<space>',
 'ฮอตพอ',
 'บาบีก้อน',
 'นิสสันหน่อย',
 'เอ็มเค',
 'ลีโอด้วย',
 'แสงโสม',
 'น้ำสิงห์',
 'mg5',
 'บาร์บี',
 'มาสด้า<space>',
 'ฮอทพอ',
 'ดีแม็ก',
 'บาบีกอ',
 'บาร์บี',
 'สิงห์',
 'บาบิ',
 'honda<space>',
 'เอมเค',
 'บาบีคิว',
 'บาบีก้อน',
 'เอ็มเค',
 'นิสสันมาร์ช',
 'บาบีก้อน',
 'เบียร์สิงห์',
 'ดีแม็ค',
 'เบียร์สิงห์',
 'ฮอตพอ',
 'เอมเค',
 'นาวาร่า',
 'บาบีก้อน',
 'บาบีกอ',
 'ฮอนด้าซีวิค',
 'รีโว่',
 'บาบีก้อน',
 'บาบิ',
 'เอมเค',
 'นาวาร่า',
 'แสงโสม',
 'มาสด้า2',
 'บาบีก้อน',
 'ฮอตพอ',
 'บาบิ',
 'มาสด้า<space>',
 'บาร์บี',
 'ยาริสเอ',
 'กินน้ำสิงห์',
 'love<space>',
 'เบียร์สิงห์',
 'บาบีคิว',
 'โคโล',
 'ลีโอๆๆๆๆไ',
 'mg3',
 'nissan<space>',
 'บาบิ',
 'มาสด้า<space>',
 'บาบีก้อน',
 'แคมรี่',
 'ไฮลัก',
 'มาสด้า<spac

In [39]:
wordlist = list(dict.fromkeys(found_word))
for word in range(len(wordlist)):
    wordlist[word] = re.sub(r'<.*?>','', wordlist[word])
    wordlist[word] = re.sub(r'#','', wordlist[word])
    

In [40]:
wordlist

['เอมเค',
 'มาสด้า3',
 'ฮอตพอ',
 'นาวาร่า',
 'เอ็มเค',
 'mg3',
 'บาบีก้อน',
 'มาสด้าสาม',
 'แคปติ',
 'บาบีกอ',
 'บาร์บี',
 'ช้าง',
 'nissan',
 'ช้างจ่าย',
 'ฮอทพอ',
 'ดีแมก',
 'ช้างวม',
 'honda',
 'นิสสันหน่อย',
 'ลีโอด้วย',
 'แสงโสม',
 'น้ำสิงห์',
 'mg5',
 'มาสด้า',
 'ดีแม็ก',
 'สิงห์',
 'บาบิ',
 'บาบีคิว',
 'นิสสันมาร์ช',
 'เบียร์สิงห์',
 'ดีแม็ค',
 'ฮอนด้าซีวิค',
 'รีโว่',
 'มาสด้า2',
 'ยาริสเอ',
 'กินน้ำสิงห์',
 'love',
 'โคโล',
 'ลีโอๆๆๆๆไ',
 'nissan',
 'แคมรี่',
 'ไฮลัก',
 'นิสสันโน๊ต',
 'mk',
 'นิสัน',
 'toyota',
 'ลีโอ55',
 'ไดโด',
 'นิสสันครับ',
 'นิสสัน',
 'ช้างตวย',
 'นิสสัน',
 'โฮกา',
 'ฟอร์ด',
 'ลีโอเบียร์',
 'มาสด้าcx',
 'นิสสันนาวา',
 'mazda',
 'ford',
 'นิสสันมอเตอร์',
 'ซีวิค',
 'ช้าง555',
 'ช้างเปน',
 'กรี๊ดดดดด',
 'ช้างกัน',
 'นิสสันอะ',
 'แอทท',
 'นิสสันเอ็กซ',
 'นิสสันอ',
 'นิสสันอัล',
 'นิสสันเอ็ก',
 'มิตซูบิชิแอ',
 'ฮอนด้า',
 'โตโยต้า',
 'ลีโอ',
 'อัลเม',
 'mg31.5',
 'นิสสัน❤️❤️❤️',
 'graduate',
 'พี่',
 ':))',
 'อย่างงี้',
 'ไฮเนเก้น',
 'แอง',
 'วัชร์',
 'ไดเอ',